# Similitud Semantica con BERT

**Descripción:** Inferencia de lenguaje natural mediante el ajuste fino del modelo BERT Y BETO.


## Introducción

Similaridad semántica es la tarea de determinar qué tan similar
dos sentencias son, en términos de lo que significan.
Este ejemplo demuestra el uso del SNLI (Stanford Natural Language Inference) para predecir la similitud semántica de sentencia con Transformers.
Afinaremos un modelo BERT que toma dos frases como entradas
y que da como resultado una puntuación de similitud para estas dos frases.

### BERT
BERT(Bidirectional Encoder Representations from Transformers),  fue uno de los principales logros recientes en el campo de la NLP(Natural Language Processing). En el momento de su lanzamiento en 2018, esta arquitectura estableció el estado del arte en el punto de referencia [GLUE](https://gluebenchmark.com/)(General Language Understanding Evaluation) que abarca numerosas tareas como:
* La respuesta a preguntas.
* El reconocimiento de entidades nombradas.
* La inferencia del lenguaje natural. 

BERT se basó en la arquitectura [Transformer](https://arxiv.org/pdf/1706.03762.pdf) al agregar el concepto de procesamiento de izquierda a derecha y de derecha a izquierda (bidireccional). También aprovechó el concepto de aprendizaje no supervisado mediante la formación previa de un modelo de lenguaje (ML) masivo en toda la Wikipedia en inglés utilizando un [Modelo de Lenguaje Enmascarado](https://www.quora.com/What-is-a-masked-language-model-and-how-is-it-related-to-BERT).

### BETO 
BETO es una iniciativa para permitir el uso de modelos previamente entrenados en BERT para tareas de PNL en español. Los autores correspondientes publicaron recientemente la biblioteca con algunos resultados.


### Referencias
* [BERT](https://arxiv.org/pdf/1810.04805.pdf)
* [BETO](https://github.com/dccuchile/beto)
* [SNLI](https://nlp.stanford.edu/projects/snli/)


## Configuración

Las secciones comentadas corresponden a los embeddings de *BETO*. Por otra parte es necesario el uso de la version(transformers==2.11.0)

In [2]:
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/tensorflow_weights.tar.gz 
#!tar -xzvf tensorflow_weights.tar.gz
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
#!mv config.json tensorflow/
#!mv vocab.txt tensorflow/
#!pip install transformers==2.11.0
import numpy as np
import pandas as pd
import tensorflow as tf
import timeit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Configuracion de parámetros

In [3]:
max_length = 128  # Longitud máxima de la oración de entrada al modelo.
batch_size = 32
epochs = 2

# Etiquetas en nuestro conjunto de datos.
labels = ["contradicción", "vinculación", "neutral"]

## Cargar los datos
Los datos se cargan de la base de datos [SNLI](https://nlp.stanford.edu/pubs/snli_paper.pdf) la cual es una colección, libremente disponible de pares de oraciones etiquetadas, escrita por humanos basada en el epígrafe de imágenes.

Esta base de datos de entrenamiento esta constituida por 550k oraciones.
Para nuestra implementación al español usamos solamente 5000 oraciones para entrenamiento, 250 para validacion y 250 de prueba.
Esta base de datos lo construimos generando archivos .txt de 250 oraciones del archivo "train" original y traduciendo usando [Google Translate](https://translate.google.com/?source=gtx&sl=en&tl=es&op=docs), debido a las limitaciones de traduccion no pudimos enriquecer nuestra base de oraciones. El set de datos original como el traducido se encuentra en [GITHUB](https://github.com/Eduardo-Moreno/DL_equipo_2) del proyecto.

In [13]:
#!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
#!tar -xvzf data.tar.gz

!curl -OL https://raw.githubusercontent.com/Eduardo-Moreno/DL_equipo_2/master/data_en.tar.gz
!curl -OL https://raw.githubusercontent.com/Eduardo-Moreno/DL_equipo_2/master/data_esp.tar.gz
!tar -xvf data_en.tar.gz
!tar -xvf data_esp.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  111k  100  111k    0     0   197k      0 --:--:-- --:--:-- --:--:--  196k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  123k  100  123k    0     0  81737      0  0:00:01  0:00:01 --:--:-- 81737
x data_en/
x data_en/valid_df.txt
x data_en/train_df.txt
x data_en/test_df.txt
x data_esp/
x data_esp/valid_esp.txt
x data_esp/test_esp.txt
x data_esp/train_esp.txt


### Exploración de los datos

Como ya lo habiamos mencionado anteriormente el dataset de entrenamiento cuenta 550k oraciones, que provienen de SNLI

In [12]:
train = pd.read_csv('data_esp/train_esp.txt', sep = ';' )
train_df = train.rename(columns={ ' oración1': 'oracion1',  ' oración2':  'oracion2'})
valid = pd.read_csv('data_esp/valid_esp.txt', sep = ';' )
valid_df = valid.rename(columns={ ' oración1': 'oracion1',  ' oración2':  'oracion2'})
test = pd.read_csv('data_esp/test_esp.txt', sep = ';' )
test_df = test.rename(columns={ ' oración1': 'oracion1',  ' oración2':  'oracion2'})

print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

Total train samples : 4998
Total validation samples: 250
Total test samples: 249


## Descripción general del conjunto de datos:

Estos son los valores de la etiqueta de "similitud" en nuestro conjunto de datos:
- Contradicción: Las frases no comparten ninguna similitud. 
- Vinculación: Las frases tienen un significado similar. 
- Neutral: Las frases son neutrales.

In [14]:
print(f"oracion1: {train_df.loc[1, 'oracion1']}")
print(f"oracion2: {train_df.loc[1, 'oracion2']}")
print(f"similitud: {train_df.loc[1, 'similitud']}")

oracion1:  Una persona en un caballo salta sobre un avión averiado.
oracion2:  Una persona está en un restaurante, pidiendo una tortilla.
similitud: contradicción


## Pre-procesamiento

In [15]:
print("Number of missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values
similitud     0
oracion1      1
oracion2     19
dtype: int64


In [16]:
print("Train Target Distribution")
print(train_df.similitud.value_counts())

Train Target Distribution
contradicción    1663
neutral          1648
vinculación      1631
implicación        28
-                   6
entailment          2
neutro              1
Name: similitud, dtype: int64


Distribution of our validation targets.

In [18]:
train_df = (
    train_df[train_df.similitud != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_df = (
    valid_df[valid_df.similitud != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

One-hot encode training, validation, and test labels.

In [19]:
train_df["label"] = train_df["similitud"].apply(
    lambda x: 0 if x == "contradicción" else 1 if x == "vinculación" else 2
)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)

valid_df["label"] = valid_df["similitud"].apply(
    lambda x: 0 if x == "contradicción" else 1 if x == "vinculación" else 2
)
y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)

test_df["label"] = test_df["similitud"].apply(
    lambda x: 0 if x == "contradicción" else 1 if x == "vinculación" else 2
)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)

## Keras Custom Data Generator
En esta seccion se genera lotes de datos.
Toma como argumentos de entrada:
* sentence_pairs: matriz de oraciones de entrada de premisas e hipótesis.
* labels: matriz de etiquetas. 
* batch_size: tamaño de lote entero.
* shuffle: boolean, si se barajan los datos. 
* include_targets: boolean, si incluir el etiquetas. 

Regresa:
* Tuplas `([input_ids, atención_mask,` token_type_ids], labels)


Ademas en esta seccion cargamos el tokenizer BERT para codificar el texto. Usaremos un modelo preentrenado "base-base-uncase", es en esta sección donde modificamos el modelo pre-entrenado [dccuchile/bert-base-spanish-wwm-uncased](https://github.com/dccuchile/beto) para cargar el modelo preentreado en español.

In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets

        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "dccuchile/bert-base-spanish-wwm-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denota el número de lotes por época.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # # Recupera el lote del índex.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # El batch de ambas oraciones son codificados juntos y separados por el token [SEP].
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convierte el batch of caracteristicas codificadas en un arreglo.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Se establece en verdadero si el generador de datos se usa para entrenamiento / validación.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Reproducción aleatoria de índices después de cada época si la reproducción aleatoria está establecida en 'True'
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


## Construcción del modelo
Se crea el modelo bajo un alcance de estrategia de distribución de esta manera nos permite el uso Bidireccional.
definiendo las ids por medio de la tokenización en BERT.
* attention_masks: indican al modelo qué tokens deben ser atendidos.
* token_type_ids: identifican diferentes secuencias en el modelo 

Se 'congela' el modelo BERT para reutilizar las funciones previamente entrenadas sin modificarlas.

* bi_lstm: Se agregan capas entrenables encima de capas congeladas para adaptar las características entrenadas previamente en los nuevos datos.

In [ ]:
import transformers
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    bert_model = transformers.TFBertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", from_pt = True)
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

### Crear generadores de datos de validación y entrenamiento

In [ ]:
train_data = BertSemanticDataGenerator(
    train_df[["oracion1", "oracion2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["oracion1", "oracion2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

## Entrenamiento del modelo de extracción de características
El entrenamiento se realiza solo para las capas superiores para realizar la "extracción de características", lo que permitirá que el modelo utilice las representaciones del modelo previamente entrenado.

In [ ]:
start = timeit.timeit()
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)
end = timeit.timeit()
print(end - start)

In [ ]:
model.save('drive/MyDrive/my_model_esp_1')

## Fine-tuning
Este paso solo debe realizarse después de que el modelo de extracción de características haya sido entrenado para la convergencia en los nuevos datos. Este es un último paso opcional donde `bert_model` se descongela y se vuelve a entrenar con una tasa de aprendizaje muy baja. Esto puede generar una mejora significativa al adaptando de forma incremental las funciones previamente entrenadas a los nuevos datos.

In [ ]:
bert_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

## Entrenamiento del modelo completo(end-to-end)

In [ ]:
start = timeit.timeit()
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)
end = timeit.timeit()
print(end - start)

## Evaluacion del modelo en el set de prueba

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["oracion1", "oracion2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

## Inferencia sobre oraciones personalizadas
Una vez entrenado el modelo podemos poner a prueba nuestro modelo de similitud sobre oraciones personalizadas. por medio la funcion "check_similarity".

In [ ]:

def check_similarity(oracion1, oracion2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba


Check results on some example sentence pairs.

In [ ]:
oracion1 = "Dos hombres mirando por la ventana de su dormitorio."
oracion2 = "Dos hombres están mirando hacia afuera."
check_similarity(oracion1, oracion2)

In [ ]:
model.save('drive/MyDrive/my_modelBETO')
#new_model = tf.keras.models.load_model('drive/MyDrive/my_model2')